# WWW-COURSERA-DOWNLOADER

Originally created by Jansen Simanullang, 2018.

My previous version of `www-coursera-downloader` dated back from 2016 is no longer working.

Sorry for the frustation of many people who have download my previous `www-coursera-downloader` and cannot use it.


## A. Requirements

### (1) Install Geckodriver
Install Geckodriver from : https://github.com/mozilla/geckodriver/releases/

#### Check Geckodriver version

In [ ]:
!echo == GECKODRIVER ==:
!geckodriver --version

### (2) Install Some Python Packages

In [104]:
!pip install -q bs4 pillow splinter jupyter_contrib_nbextensions
!jupyter nbextension enable python-markdown/main

Enabling notebook extension python-markdown/main...
      - Validating: OK


In [ ]:
from splinter import Browser
import base64, os, sys, requests, time
from bs4 import BeautifulSoup

## B. Definitions

### B. 1 Some constants

In [ ]:
# adjust loading+time to your network speed~!
loading_time = 10

In [ ]:
homepage='https://www.coursera.org'

In [ ]:
resolution = {'low':'360','med':'540','hi':'720'}
# adjust chosen_res to your liking
chosen_res = 'hi'

### B.2 Helper Function 

In [ ]:
def screenshot():
    browser.driver.save_screenshot('screenshot.png')
    from PIL import Image
    return Image.open('screenshot.png')

In [ ]:

def encode_cjson(text):
    return base64.b64encode(bytes(str(text), "utf-8"))

def save_cjson(text):
    text = encode_cjson(text)
    with open('coursera.json','wb+') as f:
        f.write(bytes(str(text),"utf-8"))
    f.close()

def read_cjson():
    import os
    if os.path.isfile('coursera.json'):
        with open('coursera.json','rb') as f:
            a = f.read()
        f.close()
    else:
        print('credential file not found')
    return str(a).split("'")[1]

def decode_cjson(text):
    return str(base64.b64decode(text)).split("'")[1]

def get_cjson():
    import json
    text = read_cjson()
    
    uid = json.loads(decode_cjson(text))
    return uid

def remove_cjson():
    import os
    if os.path.isfile('coursera.json'):
        os.remove('coursera.json')

def get_credential():
    import getpass, os
    if os.path.isfile('coursera.json'):
        username = get_cjson()['user']
        password = get_cjson()['password']
    else:
        username = input('User Name:')
        password = getpass.getpass('Password:')
        text = '{"user":"'+username+'", "password":"'+password+'"}'
        save_cjson(text)
        
    return username, password

def remove_credential():
    remove_cjson()
    print('credential removed...')
    return

def reset_credential():
    remove_cjson()
    username, password = get_credential()
    return username, password

In [ ]:
def get_mp4_url(lesson_url):
    browser.visit(homepage+lesson_url)
    print('video playing will be paused in ' + str(loading_time) + ' seconds...')
    time.sleep(loading_time)
    try:
        browser.find_by_css('span.cif-2x.cif-fw.cif-pause').click()
    except:
        time.sleep(loading_time)
        try:
            browser.find_by_css('span.cif-2x.cif-fw.cif-pause').click()
        except:
            pass
        pass
    mp4 = browser.find_by_tag('video').find_by_tag('source')['src']
    mp4 = mp4.replace('360p/',resolution[chosen_res]+'p/')
    return mp4

In [ ]:
def mp4_downloader(lesson_id, lesson_title, lesson_url):
    
    mp4 = get_mp4_url(lesson_url)
    file_name = lesson_id +'-'+ lesson_title + '.mp4'

    downloaded_size = 0
    buffer_size = 1024

    print('requesting to download...')
    r = requests.get(mp4)
    content_length = r.headers['Content-Length']
    download_size = int(content_length)
    file_exists = os.path.isfile(file_name)
    
    if file_exists:
        
        if os.path.getsize(file_name) == download_size:
            print('file already downloaded.')
            
    elif not file_exists:
        
        if r.status_code == 200:
            f = open(file_name, 'wb')
            for buffer in r.iter_content(buffer_size):
                f.write(buffer)
                downloaded_size += buffer_size
                status='\rdownloading...' + file_name + '>>> '
                status = status +str("{:.2f}".format(downloaded_size * 100. /download_size))+'%'
                sys.stdout.write(status)
                sys.stdout.flush()
            f.close()
        else:
            print('please check your connection..')
    

## C. Coursera

### Login Page

In [ ]:
browser = Browser('firefox', headless = True)

In [ ]:
browser.visit('https://www.coursera.org/?authMode=login')
screenshot()

### Insert Login Credential

In [ ]:
#reset_credential() #uncomment to reset 
username, password = get_credential()

In [ ]:
browser.visit('https://www.coursera.org/courses?authMode=login')

In [ ]:
browser.fill('email', username)
browser.fill('password', password)

In [ ]:
screenshot()

In [ ]:
buttons = browser.find_by_tag('button')
for button in buttons:
    if (button.text == 'Log in'):
        button.click()
        break
screenshot()

### LAST ACTIVE COURSES

In [ ]:
browser.visit(homepage)
section = browser.find_by_tag('section')
titles = []

for s in section:
    title = s.find_by_tag('h4').text
    titles.append(title)
    
for i, title in enumerate(titles):
    print(i+1, title)

In [ ]:
screenshot()

There are {{len(titles)}} courses visible in this section you can pick to download.

### PICK ONE OF THE COURSES and edit here

In [ ]:
# Pick one of the courses by number
#  chosen_course = int(input('Pick one of the courses by number'))
chosen_course = 1 #<--- edit here 
section[chosen_course-1].click()
course_title = titles[chosen_course-1]

 ### {{ titles[chosen_course-1] }}

In [ ]:
screenshot()

In [ ]:
lecture_homepage = browser.driver.current_url

In [ ]:
week = browser.find_by_css('div.rc-WeekCollectionNavigationItem > div')
anchors = BeautifulSoup(week.html, 'lxml').findAll('a', attrs={})
weeks = []
for a in anchors:
    weeks.append(a['href'])

weeks, len(weeks)

In [ ]:
lessons_i = []
lessons_t = []
lessons_u = []
    
for w in range(len(weeks)):
    
    print('collecting lessons title and urls...\n')    
    browser.visit(homepage + weeks[w])
    
    w = w+1
    
    print('\nVisiting week: ' + str(w))
    time.sleep(loading_time)
    module_lessons = browser.find_by_css('div.rc-ModuleLessons')
    
    print()    
    print('Week '+str(w)+' titles:')
    lessons_title = module_lessons.find_by_tag('h5')
    for j, l in enumerate(lessons_title):
        lesson_id = str(w*100+j+1).zfill(3)
        title = l.text.replace('\n',' ')
        print(lesson_id, title)
        lessons_t.append(title)
        lessons_i.append(lesson_id)
    
    print()    
    print('Week '+str(w)+' links:')    
    lessons_url = module_lessons.find_by_tag('ul')
    anchors = BeautifulSoup(lessons_url.html, 'lxml').findAll('a')

    for j, a in enumerate(anchors):
        lesson_id = str(w*100+j+1).zfill(3)
        lesson_url = a['href']
        print(lesson_id, lesson_url)
        lessons_u.append(a['href'])
        
    print()

### Download Only Videos

In [ ]:
lessons = zip(lessons_i, lessons_t, lessons_u)
for a,b,c in lessons:
    print(a,b)
    if 'Lecture' in b.split(' ')[0]:
        mp4_downloader(a,b,c)
    else:
        print('no video to download.')
    print()

## Screenshot Checking

In case of an error, you may want to check the screenshot. Here is an example:

In [ ]:
screenshot()

## Manually Calling `mp4_downloader`

In case you need to manually call `mp4_downloader` here is an example:

```
mp4_downloader('201',
               'Lecture Driving Robots Around?', 
               '/learn/mobile-robot/lecture/4HvCW/driving-robots-around')
````
`201` means week `2` lecture `01`

## Close Browser After Download

In [ ]:
browser.quit()